<a href="https://colab.research.google.com/github/Softmax20/hello-world/blob/master/get_logrank_idx%2C%20modify_Action.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install lifelines

     |████████████████████████████████| 348kB 5.1MB/s 


In [2]:
from lifelines.statistics import logrank_test
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

# 대회 데이터셋 DataFrame으로 불러오기
from google.colab import drive
drive.mount("/content/gdrive")
%cd /content/gdrive/My Drive
X_df = pd.read_csv('trainX.csv')
Y_df = pd.read_csv('trainY.csv')
A_df = pd.read_csv('trainA.csv')
test_df = pd.read_csv('testX.csv')

tabels = [X_df, Y_df, A_df, test_df]

for tabel in tabels:
  tabel.drop('Unnamed: 0', axis=1, inplace=True)

from sklearn.neighbors import NearestNeighbors

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive


In [24]:
def get_logrank_idx(X_df, Y_df, A_df, n_max, n_min, p_threshold=0.005):
    
    # n 범위와 p 기준에 따라 n별 index, 중복 index, 총 index 모음 반환
    
    Time = Y_df['time']
    Event = Y_df['event']
    n_indices = dict()
    total_indices=[]
    rep_indices=[]
    for n in range(n_min, n_max+1):

        indices = []
        
        # kneighbors: 각 trainX를 중심으로 한 n개의 인접 환자 ndarray
        neigh = NearestNeighbors(n_neighbors=n).fit(X_df)
        kneighbors = neigh.kneighbors(X_df, n_neighbors=n)[1]
        
        for i in range(0, 286):
            
            # Action에 따라 군집을 나누기 위해 인덱스 추출
            Action = A_df.iloc[kneighbors[i]]
            Action1 = Action[Action['Action']==1].index
            Action0 = Action[Action['Action']==0].index
            
            # log rank test 시행
            results = logrank_test(Time.iloc[Action1], Time.iloc[Action0], event_observed_A=Event.iloc[Action1], event_observed_B=Event.iloc[Action0])
            
            # p < p_threshold 일 경우 결과 출력 및 군집 중심 저장
            if results.p_value < p_threshold:
                indices.append(i)
                total_indices.append(i)
        
        n_indices[str(n)] = indices
        print(f'n_neighbors= {n} 일 때 index= {indices}, 총 {len(indices)}개')

    for i in total_indices:
        if total_indices.count(i)>(n_max-n_min):
            rep_indices.append(i)
    
    rep_indices = list(set(rep_indices))
    print(f'n_neighbors: {n_min}~{n_max} 일 때 중복 index= {rep_indices}, 총 {len(rep_indices)}개')
    
    total_indices = list(set(total_indices))
    print(f'n_neighbors: {n_min}~{n_max} 일 때 총 index= {total_indices}, 총 {len(total_indices)}개')
    
    return n_indices, rep_indices, total_indices

In [25]:
n_indices, rep_indices, total_indices = get_logrank_idx(X_df, Y_df, A_df, 12, 10, p_threshold=0.004)

n_neighbors= 10 일 때 index= [17, 30, 47, 90, 91, 93, 122, 126, 149, 180, 197, 237, 242, 248, 258, 272], 총 16개
n_neighbors= 11 일 때 index= [15, 30, 47, 76, 90, 105, 122, 126, 163, 180, 197, 237, 242, 248, 258, 275], 총 16개
n_neighbors= 12 일 때 index= [15, 30, 47, 76, 90, 105, 122, 126, 160, 163, 197, 237, 248, 258], 총 14개
n_neighbors: 10~12 일 때 중복 index= [258, 197, 122, 237, 47, 30, 248, 90, 126], 총 9개
n_neighbors: 10~12 일 때 총 index= [258, 15, 272, 17, 275, 149, 30, 160, 163, 47, 180, 197, 76, 90, 91, 93, 105, 237, 242, 248, 122, 126], 총 22개


In [27]:
print(n_indices)

{'10': [17, 30, 47, 90, 91, 93, 122, 126, 149, 180, 197, 237, 242, 248, 258, 272], '11': [15, 30, 47, 76, 90, 105, 122, 126, 163, 180, 197, 237, 242, 248, 258, 275], '12': [15, 30, 47, 76, 90, 105, 122, 126, 160, 163, 197, 237, 248, 258]}


In [28]:
print(rep_indices)

[258, 197, 122, 237, 47, 30, 248, 90, 126]


In [29]:
print(total_indices)

[258, 15, 272, 17, 275, 149, 30, 160, 163, 47, 180, 197, 76, 90, 91, 93, 105, 237, 242, 248, 122, 126]


In [31]:
def modify_Action(X_df, Y_df, A_df, n, indices):

    TA_df = pd.concat([Y_df['time'], A_df['Action']], axis=1, sort=False)
    new_A_df = A_df.copy()
    
    # kneighbors: 각 trainX를 중심으로 한 n개의 인접 환자 ndarray
    neigh = NearestNeighbors(n_neighbors=n).fit(X_df)
    kneighbors = neigh.kneighbors(X_df, n_neighbors=n)[1]
    
    for i in indices:
        
        Action = new_A_df.iloc[kneighbors[i]]
        Action1 = Action[Action['Action']==1].index
        Action0 = Action[Action['Action']==0].index
        
        TA_i = TA_df.iloc[kneighbors[i]]
            
        if np.mean(TA_i['time'][TA_i['Action']==1])-np.mean(TA_i['time'][TA_i['Action']==0])>0:
            new_A_df['Action'].iloc[Action0]=1
            
        else:
            new_A_df['Action'].iloc[Action1]=0
    
    return new_A_df

In [35]:
A1_df = modify_Action(X_df, Y_df, A_df, n=12, indices=rep_indices)

In [50]:
(A1_df==A_df)['Action'].value_counts()

True     651
False     15
Name: Action, dtype: int64